In [2]:
import csv
from torch.utils.data import Dataset
import torch
from sklearn.model_selection import train_test_split
import numpy as np
from bs4 import BeautifulSoup
import string
import spacy
import jsonlines
import json
import re
import torch.nn as nn
from torch.nn.utils.rnn import pad_packed_sequence,pack_padded_sequence,pad_sequence
import torch.nn.functional as F
import torch.optim as optim
import fasttext
from torch.utils.data import SubsetRandomSampler,DataLoader,Subset
from torchtext.vocab import GloVe
from tqdm import tqdm
import io



#   Which Dataset to use
#   1 = Assignment 1 Training Data
#   2 = Assignment 2 Training Data

# SENTENCE_SPLITTING_USED; whether to use the splitting of reviews into sentences.

DATASET_TO_USE = 1
SENTENCE_SPLITTING_USED = True

EMBED_DIM = 300
HIDDEN_DIM = 128
CNN_DIM = 256

PATIENCE_PARAMETER = 7
VALIDATION_LOSS_COMPUTE_STEP = 1
NUM_FILTERS = 50

device_cpu = torch.device('cpu')
device_fast = torch.device('cpu')

FILTER_SIZES = [3]

if torch.has_mps:
    device_fast = torch.device('mps')
elif torch.has_cuda:
    device_fast = torch.device('cuda')

#torch.manual_seed(0)
#np.random.seed(0)
nlp = spacy.load('en_core_web_sm')
glove = GloVe()

print(torch.cuda.is_available())

In [3]:
def preprocess_text(text):    
    text = re.sub(r'<br /><br />',".",text)
    text = BeautifulSoup(text,'lxml').get_text().strip()
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = ' '.join(re.findall(r"[\w']+|[.,!;/\"]", text))
    
    new_text = []
    for word in text.split():
        if word == '':
            continue
        new_text.append(word)
    
    text = ' '.join(new_text)
    words = nlp(text)
    text =  " ".join([token.text for token in words if not token.is_punct or token.text=='/' or token.text=="\"" or token.text=="."]).strip()
    new_words = []
    for word in text.split(" "):
        if word == 'n\'t':
            if len(new_words) > 1:
                new_words[-1] = new_words[-1] + word
            else:
                new_words.append(word)
        else:
            new_words.append(word)
    
    text = " ".join(new_words)
    return text

In [ ]:
# preprocess the training data which was given for Assignment 1
def process_assignment1_training_data():
    negative_reviews = io.open('./Train.neg',encoding='latin-1').readlines()
    positive_reviews = io.open('./Train.pos',encoding='latin-1').readlines()
    with jsonlines.open('train.jsonl',mode='w') as writer:

        for review in positive_reviews:
            processed_text = preprocess_text(review)
            d = {'text': processed_text , 'label': 1.0}
            writer.write(d)
        for review in negative_reviews:
            processed_text = preprocess_text(review)
            d = {'text': processed_text , 'label': 0.0}
            writer.write(d)

# preprocess the training data which was given for Assignment 2
def process_assignment2_training_data():
    preprocessed_dataset = []
    train_dataset_labels = []
    with open("./Train dataset.csv") as csvfile:
        csvFile = csv.reader(csvfile)
        next(csvFile)
        json_writer = jsonlines.open('processed_dataset.jsonl','w')

        for line in csvFile:
            processed_text = preprocess_text(line[0])
            label = 1.0 if line[1] == 'positive' else 0.0
            train_dataset_labels.append(label)
            json_writer.write({"text":processed_text,"label":label})
            preprocessed_dataset.append({"text":processed_text,"label":label})
    
        json_writer.close()

if DATASET_TO_USE == 1:
    process_assignment1_training_data()
else:
    process_assignment2_training_data()

In [6]:
preprocessed_dataset = []
train_dataset_labels = []


TRAIN_FILE_NAME = './train.jsonl' if DATASET_TO_USE==1 else './processed_dataset.jsonl'

with open(TRAIN_FILE_NAME ,encoding='utf-8') as f:
#with open('processed_dataset.jsonl',encoding='utf-8') as f:
    for line in f:
        sample = json.loads(line)
        train_dataset_labels.append(sample['label'])
        preprocessed_dataset.append(sample)
      
train_dataset_labels = np.array(train_dataset_labels)

NameError: name 'DATASET_TO_USE' is not defined

In [5]:
def getWordEmbeddingforText(text,glove=glove):
    length = 0
    words = []
    text = text.strip()
    for word in text.split(' '):
        length+=1
        word_embedding = glove[word]
        words.append(word_embedding)

    # If the number of words in a sentence is below 3
    if length < max(FILTER_SIZES):
        words.append(torch.zeros(EMBED_DIM))
        length+=1
    
    return torch.stack(words),length

In [6]:
def review_to_embed(review,glove=glove): 
    sentences = review.split(".")
    sentence_lengths = []
    review_embeddings = []
    num_sentences = 0
    for sentence in sentences:
        if sentence == '':
            continue
        s= sentence.strip()
        num_sentences += 1
        sentence_word_embeddings,sentence_length = getWordEmbeddingforText(s,glove)
        sentence_lengths.append(sentence_length)
        review_embeddings.append(sentence_word_embeddings)

    return torch.nn.utils.rnn.pad_sequence(review_embeddings,batch_first=True),sentence_lengths,num_sentences

In [7]:
class ReviewDataSet(Dataset):
    
    def __init__(self,reviews):
        super().__init__()
        self.reviews = reviews
        
    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, index):
        return self.reviews[index]

In [8]:
processed_dataset = []
for review in preprocessed_dataset:
    if SENTENCE_SPLITTING_USED:
        embeddings, sent_length ,n_sents = review_to_embed(review['text'])
        processed_dataset.append({'review': embeddings,'sent_lengths': sent_length,'length' : n_sents,'label' : review['label']})
    else:
        embedding,length = getWordEmbeddingforText(review['text'])
        processed_dataset.append({'review': embedding,'length': length,'label' : review['label']})

In [9]:
torch.cuda.is_available()

True

In [10]:
dataset = ReviewDataSet(processed_dataset)

In [11]:
def collate_fn_no_sentence_split(batch_data):
    inputs = [b['review'] for b in batch_data]
    lengths = [b['length'] for b in batch_data]
    labels = torch.tensor([b['label'] for b in batch_data])

    labels = labels.unsqueeze(1)
    inputs = pad_sequence(inputs,batch_first=True)
    return  {'input' : inputs , 'lengths': lengths , 'labels' : labels }



def collate_function(batch_data):   
    inputs = [b['review'] for b in batch_data]
    sent_lengths = [ b['sent_lengths'] for b in batch_data ]
    n_sentences = [ b['length'] for b in batch_data ]
    
    labels = torch.tensor([b['label'] for b in batch_data])

    labels = labels.unsqueeze(1)
    
    max_n_sentences = max([i.shape[0] for i in inputs] )
    max_n_words = max([i.shape[1] for i in inputs])

 
    processed_inputs = []
    for inp in inputs:

        t1 = torch.permute(inp,(2,1,0))
        t1 = torch.nn.functional.pad(t1,(0,max_n_sentences-inp.shape[0],0,max_n_words-inp.shape[1]))
        t1 = torch.permute(t1,(2,1,0))
        processed_inputs.append(t1)

    final_inp = torch.stack(processed_inputs)
    #inputs = pad_sequence(inputs,batch_first=True)
    return  {'input' : final_inp , 'sent_lengths': sent_lengths , 'lengths' : n_sentences ,'labels' : labels }

In [12]:
train_idx,valid_idx = train_test_split(np.arange(train_dataset_labels.shape[0]), 
    test_size=0.2,
    shuffle= True,
    stratify= train_dataset_labels,
    random_state=0
)

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)


if SENTENCE_SPLITTING_USED:
    train_dataloader = DataLoader(dataset,16,sampler=train_sampler,collate_fn=collate_function)
    valid_dataloader = DataLoader(dataset,16,sampler=valid_sampler,collate_fn=collate_function)
else:
    train_dataloader = DataLoader(dataset,64,sampler=train_sampler,collate_fn=collate_fn_no_sentence_split)
    valid_dataloader = DataLoader(dataset,64,sampler=valid_sampler,collate_fn=collate_fn_no_sentence_split)

In [13]:
import torch.nn.functional as F
class EnsembleModel(nn.Module):
    
    def __init__(self,EMBED_DIM,CNN_DIM,HIDDEN_DIM):
        super().__init__()
        self.rnn = nn.GRU(input_size = CNN_DIM,hidden_size = HIDDEN_DIM, batch_first = True)
        self.cnn = nn.Conv1d(in_channels=EMBED_DIM,out_channels=CNN_DIM,kernel_size=3)
        self.fc = nn.Linear(HIDDEN_DIM,1)


    def forward(self,inp : torch.Tensor,n_sents=None):

        ## inp  = (batch_size,max_sent_length,max_word_length,embed_dim)

        outputs = []
      
        
        for i in range(inp.shape[1]):
            current_inp = inp[:,i,:,:]
            current_inp = torch.permute(current_inp,(0,2,1))
            current_output = self.cnn(current_inp)
            current_output = F.max_pool1d(current_output,kernel_size = current_output.shape[2]).squeeze(dim=2)
            outputs.append(current_output)
        
        #print(len(outputs))
        #print(outputs[0].shape)
        lstm_in = torch.stack(outputs,dim=1)
   
        packed_input = pack_padded_sequence(lstm_in,n_sents,batch_first=True,enforce_sorted=False)
        packed_output,hidden = self.rnn(packed_input)
        output,output_lengths = pad_packed_sequence(packed_output,batch_first=True)

        hidden = torch.permute(hidden,(1,0,2))
        hidden = hidden.contiguous().view((hidden.shape[0],-1))

        out = self.fc(hidden)
        return nn.Sigmoid()(out)
        #out = self.cnn(inp)
        #return out

batch_data = next(iter(train_dataloader))
j = EnsembleModel(EMBED_DIM,CNN_DIM,HIDDEN_DIM)
j(batch_data['input'],batch_data['n_sent'])

tensor([[0.5234],
        [0.5198],
        [0.5229],
        [0.5133],
        [0.5086],
        [0.5242],
        [0.5163],
        [0.5377],
        [0.5196],
        [0.5319],
        [0.5277],
        [0.5291],
        [0.5295],
        [0.5129],
        [0.5254],
        [0.5222]], grad_fn=<SigmoidBackward0>)

In [14]:
import torch.nn.functional as F
class CNNLSTMAttention(nn.Module):
    
    def __init__(self,EMBED_DIM,CNN_DIM,HIDDEN_DIM,bidirectional_factor = 2,fc_dropout=0.3):
        super().__init__()

        bidirectional = False

        if bidirectional_factor==2:
            bidirectional = True

        self.rnn = nn.GRU(input_size = CNN_DIM,hidden_size = HIDDEN_DIM, bidirectional=bidirectional,batch_first = True)
        self.cnn = nn.Conv1d(in_channels=EMBED_DIM,out_channels=CNN_DIM,kernel_size=3)
        self.attention_layer = nn.Linear(bidirectional_factor* HIDDEN_DIM,1)
        self.fc = nn.Linear(bidirectional_factor*HIDDEN_DIM,HIDDEN_DIM)
        self.fc_dropout = nn.Dropout(fc_dropout)
        self.out_fc = nn.Linear(HIDDEN_DIM,1)
        self.batchnorm1d = nn.BatchNorm1d(CNN_DIM)

    def forward(self,inp : torch.Tensor,n_sents=None):

        ## inp  = (batch_size,max_sent_length,max_word_length,embed_dim)

        outputs = []
      
        
        for i in range(inp.shape[1]):
            current_inp = inp[:,i,:,:]
            current_inp = torch.permute(current_inp,(0,2,1))
            current_output = self.cnn(current_inp)
            current_output = F.max_pool1d(current_output,kernel_size = current_output.shape[2]).squeeze(dim=2)
            outputs.append(current_output)
        
        #print(len(outputs))
        #print(outputs[0].shape)
        lstm_in = torch.stack(outputs,dim=2)
        lstm_in = self.batchnorm1d(lstm_in)
        lstm_in = torch.permute(lstm_in,(0,2,1))

        packed_input = pack_padded_sequence(lstm_in,n_sents,batch_first=True,enforce_sorted=False)
        packed_output,hidden = self.rnn(packed_input)
        output,output_lengths = pad_packed_sequence(packed_output,batch_first=True)
        attention_logs = self.attention_layer(output).squeeze(dim=2)
        attention_score = F.softmax(attention_logs,dim=1).unsqueeze(2)

        final_out = attention_score*output

        averaged_vector = torch.sum(final_out,dim=1,keepdim=False)

        #hidden = torch.permute(hidden,(1,0,2))
        #hidden = hidden.contiguous().view((hidden.shape[0],-1))
        out = self.fc_dropout(F.leaky_relu(self.fc(averaged_vector)))
        out = self.out_fc(out)
        return nn.Sigmoid()(out)
        #out = self.cnn(inp)
        #return out


In [15]:
import os
from torch.utils.tensorboard import SummaryWriter
from datetime import  datetime

def train(model,train_dataloader,valid_dataloader,num_epochs,criterion,optimizer,
    checkpoint_name='best_model.pt',
    device_train = device_fast,use_rnn = False,log=True):

    tensorboard_name='Ensemble'
    if log == True:
        current_datetime = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
        tensorboard_name = tensorboard_name + "_" + current_datetime
        writer = SummaryWriter('runs/' + tensorboard_name)
    
    
    model = model.to(device_train)
    clip = 0
    if use_rnn:
        clip = 5

    best_validation_loss = 1000.0
    valdiation_loss_not_decreased_steps = 0
    
    model.train()
    for e in range(num_epochs):
        
        training_set_size = 0
        training_loss = 0.0
        model.train()

        for data in tqdm(train_dataloader):
            
            optimizer.zero_grad()
            if SENTENCE_SPLITTING_USED:
                input_reviews,sent_lengths,n_sents,output_labels = data['input'], data['sent_lengths'],data['lengths'],data['labels']
            else:
                input_reviews,n_sents,output_labels = data['input'],data['lengths'],data['labels']

            input_reviews = input_reviews.to(device_train)
            training_set_size += input_reviews.shape[0]
            output = model(input_reviews,n_sents)
            output = output.to(device_cpu)
            loss = criterion(output,output_labels.float())
            training_loss += loss.item()
            loss.backward()
            if use_rnn:
                nn.utils.clip_grad_norm_(model.parameters(),clip)
            optimizer.step()
        
        current_training_loss = training_loss
        if log==True:
            print("Epoch " + str(e) + " Average Training Loss = " +  str(current_training_loss))
            writer.add_scalars(tensorboard_name + 'Training Loss vs Epoch',{'train' : current_training_loss},e)

        
        model.eval()
        
        if valid_dataloader is None:
            continue
        
        validation_set_size  = 0 
        if e% VALIDATION_LOSS_COMPUTE_STEP==0:
            correct_count = 0
            validation_loss = 0

            for i,data in enumerate(valid_dataloader,0):
                if SENTENCE_SPLITTING_USED:
                    input_reviews,sent_lengths,n_sents,output_labels = data['input'], data['sent_lengths'],data['lengths'],data['labels']
                else:
                    input_reviews,n_sents,output_labels = data['input'],data['lengths'],data['labels']
                
                input_reviews = input_reviews.to(device_train)
                validation_set_size += input_reviews.shape[0]
                output = model(input_reviews,n_sents)
                output = output.to(device_cpu)
                loss = criterion(output,output_labels.float())
                validation_loss += loss.item()
                nearest_class = torch.round(output)

                correct = (nearest_class == output_labels.float()).float()
                correct_count += correct.sum()
            correct_count = int(correct_count)
            current_validation_accuracy = (correct_count/validation_set_size)*100
            current_validation_loss = (1.0* validation_loss)
            if log == True:
                print("Epoch " + str(e) + " " +  "Validation Loss = " + str(current_validation_loss) )
                print("Validation Set Accuracy = " + str((correct_count/validation_set_size)*100) )
                writer.add_scalar(tensorboard_name + ' Validation Accuracy vs Epoch ',(correct_count/validation_set_size*100),e)
                writer.add_scalars(tensorboard_name + 'Validation Loss vs Epoch',{'valid' : current_validation_loss},e)

            
            if log==True:
                if current_validation_loss < best_validation_loss:
                    valdiation_loss_not_decreased_steps = 0
                    torch.save(model.state_dict(),checkpoint_name)
                    best_validation_loss = current_validation_loss
                else:
                    valdiation_loss_not_decreased_steps +=1
        if log == True:
            if valdiation_loss_not_decreased_steps >= PATIENCE_PARAMETER:
                break

In [16]:
torch.cuda.empty_cache()

net = CNNLSTMAttention(EMBED_DIM,CNN_DIM,HIDDEN_DIM)
optimizer= optim.SGD(net.parameters(),lr=0.0054,momentum=0.9,nesterov=True)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0054, max_lr=0.0072,step_size_up=10000)
train(net,train_dataloader,valid_dataloader,100,nn.BCELoss(),optimizer,'first_cnn_rnn_att_adam_batch_nrom_cyclelr_bidir_0.0054.pt',device_fast,True,True)

100%|██████████| 500/500 [00:17<00:00, 28.89it/s]


Epoch 0 Average Training Loss = 304.7490390241146
Epoch 0 Validation Loss = 73.76824501156807
Validation Set Accuracy = 67.7


100%|██████████| 500/500 [00:10<00:00, 46.12it/s]


Epoch 1 Average Training Loss = 228.81198932230473
Epoch 1 Validation Loss = 61.37838926911354
Validation Set Accuracy = 74.9


100%|██████████| 500/500 [00:07<00:00, 68.15it/s]


...

Epoch 6 Average Training Loss = 60.15787902229931
Epoch 6 Validation Loss = 106.82008984684944
Validation Set Accuracy = 75.3


100%|██████████| 500/500 [00:07<00:00, 67.40it/s]


Epoch 7 Average Training Loss = 50.93568859464722
Epoch 7 Validation Loss = 117.82814064621925
Validation Set Accuracy = 75.14999999999999


100%|██████████| 500/500 [00:07<00:00, 66.01it/s]


Epoch 8 Average Training Loss = 48.31048094131984
Epoch 8 Validation Loss = 109.44376400113106
Validation Set Accuracy = 77.0


In [17]:
torch.cuda.empty_cache()
net = EnsembleModel(EMBED_DIM,CNN_DIM,HIDDEN_DIM)
train(net,train_dataloader,valid_dataloader,50,nn.BCELoss(),optim.Adam(net.parameters(),0.001),'cnn_rnn_dnn_adam.pt',device_fast,True,True)

  0%|          | 0/2000 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 46.00 MiB (GPU 0; 2.00 GiB total capacity; 1.64 GiB already allocated; 0 bytes free; 1.70 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
def test(model_name,test_data,test_lengths,test_labels):
    model = CNNLSTMAttention(EMBED_DIM,CNN_DIM,HIDDEN_DIM)
    model.load_state_dict(torch.load(model_name,map_location=device_cpu))
    model.eval()
    count = 0
    for i in range(len(test_data)):
        ans = model(test_data[i],[test_lengths[i]])
        ans = torch.round(ans)
        if ans[0][0] == test_labels[i]:
            count+=1
    
    print("Accuracy = " + str((count/len(test_data)*100)))


In [19]:
test_word_embeddings = [] 
test_sentence_lengths = []
test_dataset_labels = []  

def getAssignment2TestData():
    test_processed_text = []
    with open("./E0334 Assignment2 Test Dataset.csv",encoding='utf-8') as csvfile:
        csvFile = csv.reader(csvfile)
        next(csvFile)
        for line in csvFile:
            processed_text = preprocess_text(line[0])
            label = 1.0 if line[1] == 'positive' else 0.0
            test_dataset_labels.append(label)
            test_processed_text.append(processed_text)

    for i in range(len(test_processed_text)):
        if SENTENCE_SPLITTING_USED:
            current_embeddings,current_sent_lengths,current_n_sent = review_to_embed(test_processed_text[i]) 
        else:
            current_embeddings,current_n_sent = getWordEmbeddingforText(test_processed_text[i])

        test_word_embeddings.append(current_embeddings.clone().detach().unsqueeze(0))
        test_sentence_lengths.append(current_n_sent)


def getAssignment1TestData():
    
    correct_count = 0

    reviews = open('./TestData','r',encoding='latin-1').readlines()
    for i in range(len(reviews)):
        r = reviews[i]
        reviews[i] = preprocess_text(r)

        if SENTENCE_SPLITTING_USED:
            current_embeddings,current_sent_lengths,current_n_sent = review_to_embed(reviews[i]) 
        else:
            current_embeddings,current_n_sent = getWordEmbeddingforText(reviews[i])
      
        if (i<331):
            test_dataset_labels.append(1.0)
        else:
            test_dataset_labels.append(0.0)
        test_word_embeddings.append(current_embeddings.clone().detach().unsqueeze(0))
        test_sentence_lengths.append(current_n_sent)

c:\Users\Vivitsu329\Documents\environments\nlpenv\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [21]:
test('./cnn_rnn_att_adam_batch_nrom_cyclelr_bidir_0.0054.pt',test_word_embeddings,test_sentence_lengths,test_dataset_labels)

Accuracy = 89.78897889788979


In [22]:
test('./first_cnn_rnn_att_adam_batch_nrom_cyclelr_bidir_0.0054.pt',test_word_embeddings,test_sentence_lengths,test_dataset_labels)

Accuracy = 80.76923076923077
